# Safe Run
Made to get around Discord Cloudflare error that was preventing bots from getting online. The inputs are the bot object and the filename. The filename is input so that any this function can be used in any design. When the error comes up,the function uses kill 1 to stop all processes in repl then starts bot.py with `python bot.py`. Potentially, if the error message was to change this would stop working but it would not be hard to make it work again by changing `expectederrmsg`.

In [ ]:
def saferun(bot,bfilename):
    import os
    expectederrmsg="The owner of this website (discord.com) has banned you temporarily from accessing this website"
    # Normal Boot
    TOKEN = os.environ.get("DISCORD_BOT_SECRET") 
    try:
        bot.run(TOKEN)
        
    #Safety Mechanism
    except Exception as e:
        exceptionmessage = str(e)
        if (expectederrmsg
                in exceptionmessage):
           

            print("Shuttting down to restart")
            #kill all processes then start bot file again
            os.system(f"kill 1 |python {bfilename}.py")

# Remote Start V1

   Still a bit buggy but not bad for a new development. It works by booting a flask webserver as main.py. When someone goes to the website, the `count` variable is incremented. There was no real logic behind this just trial and error but the bot boots when `count` is equal to 2. I suppose I should refine this in the future but it works(somewhat). I don't know what is causing it exactly but multiple instances of the bot are booting which is not a terrible problem but is a quirky bug. I plan to see if I can utilize this little bug to my advantage but experiments to cause it so far have been unsucessful. Anyhow, once `count` is 2, `start.sh` is called which just starts the bot. I could have done `os.system("python bot.py")` as well but it works. `count` is also reset back to 0.

In [ ]:
from flask import Flask

global count
count = 0

app = Flask('')
app = Flask(__name__)


@app.route('/')  # this is the home page route
@app.route('/boot', methods=['POST'])
def boot():

    global count
    import subprocess
    print("recieved call")
    print(count)
    count += 1
    if (count == 2):
        count = 0  #reset count

        subprocess.call(['sh', 'start.sh']) #optionally os.system("python bot.py")
        return "booted"
    return "Swift"


def run():
    app.run(host="0.0.0.0", port=8080)


run()


![Puzzle](watchboot.png)  

# Suit Power Standby 
 Puts bot on standby if suit user goes offline.  
 `powermanip` changes the bots status. If 0 is inputted, the bot will go offline and if 1 is inputted it will go online
 `on_member_update` checks the status of the user and calls powermanip as needed

In [ ]:
global standby #so bot doesnt repeat boots
standby=False 
async def powermanip(switch):
    if(switch):
      await bot.change_presence(status=discord.Status.online)
      
    if(not switch):
     await bot.change_presence(status=discord.Status.offline)


    
  
@bot.event
async def on_member_update(before, after):
    global suituser
    global standby
    if(after.id==suituser and str(before.status)!="offline" and str(after.status)=="offline"):
     standby=True
     await powermanip(0)
    elif(after.id==suituser and str(before.status)!="online" and str(after.status)=="online" and standby==True):
      standby=False
      await powermanip(1)

# Targeting System V4

In [ ]:
#Target Parse V4
#Takes in ctx: command context
#user string or int
async def targetparse(ctx, user):
    user = user.strip()  #eliminate whitespace
    target = ""  #target holder
    #print(target)   #Debug
    #check if user is a String
    if ((user.replace(" ", "").isalpha())):
        #check if target is in guild members
        for member in ctx.guild.members:
            if ((user == (member.display_name)) or (user == (member.nick))
                    or (user == (member.name))):
                target = member
                return target
    #check if its a ping mention
    elif ("@" in user):

        memid = int(user.replace("<@", " ").replace(">", " "))
        target = await bot.fetch_user(memid)
        return target
    #check if user is an int
    elif (user.isdigit()):
        target = await bot.fetch_user(user)
        return target


# Saturn Install
  Small but powerful. Speeds up boot time by installing nonessential libraries on the fly. Named after Saturn V rocket because of how the rocket worked in stages.


In [ ]:
import subprocess
def saturninstall(pack):
    subprocess.call(['pip', 'install', pack])

# Suit Basic Init

Made to initialize bot variables from a json file to allow for faster suit construction and also allow the variables to be kept in their state during last use if repl restarts the bot. 

Sample Init File
```json
{
  
    "suituser": 590010502781075459,
    "prefix": "*",
    "weapons": false,
    "entrygif":"",
    "weplist" : [{
        "name":"repulsor",
        "type":"fire",
        "gif":"https://tenor.com/view/war-machine-gun-up-gif-19585303"
    }],
    "wepnum":0
    
  }


```

SBCV1 Suit Basic Controller V1: Intiates Suit Variables from Init File

In [ ]:
import json
def suitbasicinit():
    #Load in suit settings
    global settings
    global suituser
    global weapons
    global wepnum
    global weplist

    suitsettings = open("suit.json")
    settings = json.load(suitsettings)
    suituser = settings['suituser']
    weapons = settings['weapons']
    weplist = settings['weplist']


#Suit Basics Controller V1
#needed to manipulate files
import json
#discord classes
import discord
#Firebase imports
import firebase_admin
from firebase_admin import credentials, db

#Suit class to define common variables and suit functions
class Suit:
    suituser = 0
    prefix = ""
    weapons = False
    state = None
    entrygif = ""
    cogs = []
    botvar = None
    datab = None

    def load(self, bot, filename="suitvar.json"):
        #open json file
        suitsettings = open(filename)
        #load json file
        settings = json.load(suitsettings)
        #get userid of suituser
        self.suituser = int(settings['suituser'])
        #get bot prefix
        self.prefix = settings['prefix']
        #get weapon boolean
        self.weapons = settings['weapons']
        #get and set suit state
        state = SuitState()
        self.state = state.loadstate(settings['state'])
        #get entry gif
        self.entrygif = settings['entrygif']
        #get cogs for loading
        self.cogs = settings['cogs']
        #passing bot to botvar
        self.botvar = bot
        #set the prefix
        self.botvar.command_prefix = self.prefix
        #load db
        credfile, dburl = settings['dbdata']
        self.datab = Database(credfile, dburl)

        #load suit cogs
        for cog in self.cogs:
            try:
                #load cog by cog
                bot.load_extension(f"Cogs.{cog}")
            except Exception as e:
                #print exception if the cog file has an error
                print(e)

    #do entry function
    async def entry(self, guild):
        #check if there is a guild system channel to send gif too
        if guild.system_channel:
            await guild.system_channel.send(file=discord.File(self.entrygif))

    #send message text to user
    #messagetxt: text input
    #mode:format of input
    async def usersend(self, messagetext, mode="text"):

        user = await self.botvar.fetch_user(self.suituser)

        if (mode == "text"):

            await user.send(messagetext)
        if (mode == "file"):
            await user.send(file=discord.File(messagetext))

#Suit State 
class SuitState:
    name = ""
    variablesset = {}
    triggers = {}

    def loadstate(self, jsonin):
        state = jsonin.split(" ")
        self.name = state[0]
        self.variablesset = {}
        self.triggers = {}

#Database Class
class Database:
    url = ""
    credfile = ""
    dbobj = None
     #initialze the db from the credential file and the url 
    def __init__(self, credfile, url):

        cred = credentials.Certificate(credfile)

        firebase_admin.initialize_app(cred, {'databaseURL': url})

        self.dbobj = db
    #IO operations
    #path database path to read or write too
    #mode read or write Defaults to read
    #writearg Optional what to write, if nothing written does not need to be sent
    def io(self, path, mode="r", *writearg):

        if (mode == "r"):

            ref = self.dbobj.reference(path)
            data = ref.get()
            return data
        elif (mode == "w"):
            ref = self.dbobj.reference(path)
            ref.set(writearg)
#Suit Basics Controller V1
#needed to manipulate files
import json
#discord classes
import discord
#Firebase imports
import firebase_admin
from firebase_admin import credentials, db

#Suit class to define common variables and suit functions
class Suit:
    suituser = 0
    prefix = ""
    weapons = False
    state = None
    entrygif = ""
    cogs = []
    botvar = None
    datab = None

    def load(self, bot, filename="suitvar.json"):
        #open json file
        suitsettings = open(filename)
        #load json file
        settings = json.load(suitsettings)
        #get userid of suituser
        self.suituser = int(settings['suituser'])
        #get bot prefix
        self.prefix = settings['prefix']
        #get weapon boolean
        self.weapons = settings['weapons']
        #get and set suit state
        state = SuitState()
        self.state = state.loadstate(settings['state'])
        #get entry gif
        self.entrygif = settings['entrygif']
        #get cogs for loading
        self.cogs = settings['cogs']
        #passing bot to botvar
        self.botvar = bot
        #set the prefix
        self.botvar.command_prefix = self.prefix
        #load db
        credfile, dburl = settings['dbdata']
        self.datab = Database(credfile, dburl)

        #load suit cogs
        for cog in self.cogs:
            try:
                #load cog by cog
                bot.load_extension(f"Cogs.{cog}")
            except Exception as e:
                #print exception if the cog file has an error
                print(e)

    #do entry function
    async def entry(self, guild):
        #check if there is a guild system channel to send gif too
        if guild.system_channel:
            await guild.system_channel.send(file=discord.File(self.entrygif))

    #send message text to user
    #messagetxt: text input
    #mode:format of input
    async def usersend(self, messagetext, mode="text"):

        user = await self.botvar.fetch_user(self.suituser)

        if (mode == "text"):

            await user.send(messagetext)
        if (mode == "file"):
            await user.send(file=discord.File(messagetext))

#Suit State 
class SuitState:
    name = ""
    variablesset = {}
    triggers = {}

    def loadstate(self, jsonin):
        state = jsonin.split(" ")
        self.name = state[0]
        self.variablesset = {}
        self.triggers = {}

#Database Class
class Database:
    url = ""
    credfile = ""
    dbobj = None
     #initialze the db from the credential file and the url 
    def __init__(self, credfile, url):

        cred = credentials.Certificate(credfile)

        firebase_admin.initialize_app(cred, {'databaseURL': url})

        self.dbobj = db
    #IO operations
    #path database path to read or write too
    #mode read or write Defaults to read
    #writearg Optional what to write, if nothing written does not need to be sent
    def io(self, path, mode="r", *writearg):

        if (mode == "r"):

            ref = self.dbobj.reference(path)
            data = ref.get()
            return data
        elif (mode == "w"):
            ref = self.dbobj.reference(path)
            ref.set(writearg)
